In [3]:
from data.processing import ParseBalanced

directory = "large-melspec-dataset-top-50-LIBROSA-256-Triplet"
data_directory = "E:/mtg-jamendo/"
subset_file_name = "autotagging_top50tags"
ParseBalanced(subset_file_name, f"{data_directory}", f"E:/SongsDataset/{directory}", convert=True, target_per_genre=1300, chunk_size=256, chunks_per_batch=1, write_individually=True)

KeyboardInterrupt: 

In [1]:
from torchaudio.transforms import TimeMasking, FrequencyMasking
#from loss.ConstrastiveLoss import InfoCNELoss
from info_nce import InfoNCE
from data.data_utils import *

directory = "large-melspec-dataset-top-50-LIBROSA-256-Triplet"
data_directory = "E:/mtg-jamendo/"
subset_file_name = "autotagging_top50tags"

augmentations = Compose([
    AddGaussianNoise(std=0.25),
    TimeMasking(time_mask_param=int(0.05* 256)),
    FrequencyMasking(freq_mask_param=int(0.05 * 128)),
])

class Config:
    # === General ===
    model_name = "ViT-Contrastive-Custom-Slope"
    device = "cuda" if torch.cuda.is_available() else "cpu"
    dtype = torch.float32
    save_path = f"trained_models\\{model_name}\\"
    seed = 42

    # === Training ===
    num_classes = 50
    num_epochs = 10
    batch_size = 64
    max_batch_size = 64
    learning_rate = 1e-4
    min_learning_rate = 1e-4
    weight_decay = 1e-4

    warmup_threshold = 1.0 / 100.0
    step_coefficient = 25.0 / 100.0

    gamma = 2.0
    save_checkpoints = True

    # === Dataset ===
    transforms = None
    use_masks = True
    num_workers = 1
    prefetch_factor = 3
    val_split = 0.1
    shuffle = True
    pos_weight = (torch.ones(num_classes) * 50).to("cuda")
    criterion = InfoNCE()

In [2]:
from torch.utils.data import DataLoader

large_directory = directory

train_dataset = StreamViewDataset(f"E:\\SongsDataset\\{large_directory}\\train_set\\data", f"E:\\SongsDataset\\{large_directory}\\train_set\\genre_labels")
test_dataset  = StreamViewDataset(f"E:\\SongsDataset\\{large_directory}\\test_set\\data", f"E:\\SongsDataset\\{large_directory}\\test_set\\genre_labels")

train_dataloader = DataLoader(
    train_dataset,
    batch_size=Config.batch_size,
    shuffle=True,
    num_workers=Config.num_workers,
    prefetch_factor=Config.prefetch_factor,
)

test_dataloader = DataLoader(
    test_dataset,
    batch_size=Config.batch_size,
    shuffle=True,
    num_workers=Config.num_workers,
    prefetch_factor=Config.prefetch_factor,
)

In [3]:
from models.AudioTransformer import AudioTransformer
from utils import misc

model = AudioTransformer(latent_space=32, input_dim=128, d_model=256, dim_feedforward=512, length=256, num_heads=8, encoder_layers=16, decoder_layers=16, dropout=0.1, use_alibi=True, custom_slopes=True)

print(f"{misc.model_size(model)} Parameters")

25462368 Parameters


In [6]:
from models.AudioViT import AudioViT
from utils import misc

model = AudioViT(latent_space=32, input_dim=128, d_model=256, length=256, num_heads=8, encoder_layers=8, decoder_layers=8, dropout=0.1, use_alibi=True, use_rope=True)

print(f"{misc.model_size(model)} Parameters")

23554720 Parameters


In [ ]:
from models.AudioPreCNNTransformer import AudioPreCNNTransformer
from utils import misc

model = AudioPreCNNTransformer(latent_space=512, input_dim=128, length=1024, num_heads=8, transformer_layers=8, d_model=256, dropout=0.1)
print(f"{misc.model_size(model)} Parameters")

In [ ]:
from models.AudioTransformerWeaved import AudioTransformerWeaved
from utils import misc

model = AudioTransformerWeaved(latent_space=128, input_dim=128, d_model=256, dim_feedforward=512, length=1024, num_heads=8, encoder_layers=8, decoder_layers=8, dropout=0.1, use_alibi=True)

print(f"{misc.model_size(model)} Parameters")

In [9]:
from models.AudioViTEncoder import AudioViTEncoder
from utils import misc

model = AudioViTEncoder(latent_space=32, input_dim=128, d_model=256, length=256, num_heads=8, encoder_layers=8, decoder_layers=8, dropout=0.1, use_alibi=True, use_rope=True)

print(f"{misc.model_size(model)} Parameters")

13792256 Parameters


In [10]:
from training.contrastive_training import train_contrastive
train_contrastive(model, test_dataloader, train_dataloader, Config, show_graph=False)

  0%|          | 0/155 [00:03<?, ?it/s]

RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x128 and 256x128)

In [3]:
from models.AudioTransformerEmbeddings import AudioTransformerEmbeddings
from utils import misc

model = AudioTransformerEmbeddings(latent_space=128, input_dim=128, d_model=256, dim_feedforward=512, length=256, num_heads=8, encoder_layers=8, dropout=0.1, use_alibi=True, use_rope=True, custom_slope=1)

print(f"{misc.model_size(model)} Parameters")

24468914 Parameters


In [4]:
from training.contrastive_training import train_contrastive
train_contrastive(model, test_dataloader, train_dataloader, Config, show_graph=False)

  0%|          | 0/155 [00:03<?, ?it/s]

Batch Loss [1/155]: 4.1884

Batch Loss [2/155]: 4.1526

Batch Loss [3/155]: 4.1499

Batch Loss [4/155]: 4.1457

Batch Loss [5/155]: 4.1451

Batch Loss [6/155]: 4.1244

Batch Loss [7/155]: 4.0774

Batch Loss [8/155]: 3.9222

Batch Loss [9/155]: 4.3615

Batch Loss [10/155]: 4.0819

Batch Loss [11/155]: 4.1155

Batch Loss [12/155]: 4.1398

Batch Loss [13/155]: 4.1480

Batch Loss [14/155]: 4.1465

Batch Loss [15/155]: 4.1500

Batch Loss [16/155]: 4.1527

Batch Loss [17/155]: 4.1539

Batch Loss [18/155]: 4.1544

Batch Loss [19/155]: 4.1559

Batch Loss [20/155]: 4.1561

Batch Loss [21/155]: 4.1558

Batch Loss [22/155]: 4.1562

Batch Loss [23/155]: 4.1552

Batch Loss [24/155]: 4.1561

Batch Loss [25/155]: 4.1577

Batch Loss [26/155]: 4.1569

Batch Loss [27/155]: 4.1568

Batch Loss [28/155]: 4.1564

Batch Loss [29/155]: 4.1559

Batch Loss [30/155]: 4.1561

Batch Loss [31/155]: 4.1563

Batch Loss [32/155]: 4.1561

Batch Loss [33/155]: 4.1565

Batch Loss [34/155]: 4.1563

Batch Loss [35/155]: 4.

  0%|          | 0/18 [00:00<?, ?it/s]

[Epoch 0] Train:  4.0232
Test:  Contrastive Loss: 3.7577


  0%|          | 0/155 [00:03<?, ?it/s]

...

[Epoch 8] Train:  2.6642
Test:  Contrastive Loss: 2.5575


  0%|          | 0/155 [00:03<?, ?it/s]

Batch Loss [1/155]: 2.3081

Batch Loss [2/155]: 2.4887

Batch Loss [3/155]: 2.4743

Batch Loss [4/155]: 2.4905

Batch Loss [5/155]: 2.8146

Batch Loss [6/155]: 2.7982

Batch Loss [7/155]: 2.9638

Batch Loss [8/155]: 2.4079

Batch Loss [9/155]: 2.5997

Batch Loss [10/155]: 2.7343

Batch Loss [11/155]: 2.5180

Batch Loss [12/155]: 2.6350

Batch Loss [13/155]: 2.9264

Batch Loss [14/155]: 2.8807

Batch Loss [15/155]: 2.4665

Batch Loss [16/155]: 2.7810

Batch Loss [17/155]: 2.1995

Batch Loss [18/155]: 2.5400

Batch Loss [19/155]: 2.9568

Batch Loss [20/155]: 2.1293

Batch Loss [21/155]: 2.4708

Batch Loss [22/155]: 2.2857

Batch Loss [23/155]: 2.6717

Batch Loss [24/155]: 2.2578

Batch Loss [25/155]: 2.4573

Batch Loss [26/155]: 2.4425

Batch Loss [27/155]: 2.5966

Batch Loss [28/155]: 2.8571

Batch Loss [29/155]: 2.5796

Batch Loss [30/155]: 2.2298

Batch Loss [31/155]: 2.4132

Batch Loss [32/155]: 2.4424

Batch Loss [33/155]: 2.6939

Batch Loss [34/155]: 2.5359

Batch Loss [35/155]: 3.

  0%|          | 0/18 [00:00<?, ?it/s]

[Epoch 9] Train:  2.5832
Test:  Contrastive Loss: 2.5203


In [7]:
from training.autoencoding_training import train_autoencode
train_autoencode(model, test_dataloader, train_dataloader, Config, show_graph=False)

  0%|          | 0/98 [00:00<?, ?it/s]

Mid-Epoch Loss [1/98]: 1.855

Mid-Epoch Loss [2/98]: 1.814

Mid-Epoch Loss [3/98]: 1.78

Mid-Epoch Loss [4/98]: 1.751

Mid-Epoch Loss [5/98]: 1.729

Mid-Epoch Loss [6/98]: 1.707

Mid-Epoch Loss [7/98]: 1.689

Mid-Epoch Loss [8/98]: 1.675

Mid-Epoch Loss [9/98]: 1.66

Mid-Epoch Loss [10/98]: 1.648

Mid-Epoch Loss [11/98]: 1.635

Mid-Epoch Loss [12/98]: 1.623

Mid-Epoch Loss [13/98]: 1.611

Mid-Epoch Loss [14/98]: 1.6

Mid-Epoch Loss [15/98]: 1.589

Mid-Epoch Loss [16/98]: 1.577

Mid-Epoch Loss [17/98]: 1.566

Mid-Epoch Loss [18/98]: 1.556

Mid-Epoch Loss [19/98]: 1.546

Mid-Epoch Loss [20/98]: 1.536

Mid-Epoch Loss [21/98]: 1.527

Mid-Epoch Loss [22/98]: 1.517

Mid-Epoch Loss [23/98]: 1.507

Mid-Epoch Loss [24/98]: 1.499

Mid-Epoch Loss [25/98]: 1.49

Mid-Epoch Loss [26/98]: 1.481

Mid-Epoch Loss [27/98]: 1.473

Mid-Epoch Loss [28/98]: 1.465

Mid-Epoch Loss [29/98]: 1.457

Mid-Epoch Loss [30/98]: 1.449

Mid-Epoch Loss [31/98]: 1.442

Mid-Epoch Loss [32/98]: 1.434

Mid-Epoch Loss [33/98]

  0%|          | 0/11 [00:00<?, ?it/s]

[Epoch 0] Train:  1.1093
Test:  Cos: 0.4985, MSE: 0.7842


In [4]:
from training.autoencoding_training import train_autoencode
train_autoencode(model, test_dataloader, train_dataloader, Config, show_graph=False)

  0%|          | 0/98 [00:00<?, ?it/s]

Mid-Epoch Loss [1/98]: 0.7627

Mid-Epoch Loss [2/98]: 0.7602

Mid-Epoch Loss [3/98]: 0.6974

Mid-Epoch Loss [4/98]: 0.6474

Mid-Epoch Loss [5/98]: 0.6145

Mid-Epoch Loss [6/98]: 0.5882

Mid-Epoch Loss [7/98]: 0.5669

Mid-Epoch Loss [8/98]: 0.551

Mid-Epoch Loss [9/98]: 0.535

Mid-Epoch Loss [10/98]: 0.5224

Mid-Epoch Loss [11/98]: 0.5126

Mid-Epoch Loss [12/98]: 0.5031

Mid-Epoch Loss [13/98]: 0.4956

Mid-Epoch Loss [14/98]: 0.4897

Mid-Epoch Loss [15/98]: 0.4836

Mid-Epoch Loss [16/98]: 0.4773

Mid-Epoch Loss [17/98]: 0.4733

Mid-Epoch Loss [18/98]: 0.4682

Mid-Epoch Loss [19/98]: 0.4636

Mid-Epoch Loss [20/98]: 0.4598

Mid-Epoch Loss [21/98]: 0.4568

Mid-Epoch Loss [22/98]: 0.4535

Mid-Epoch Loss [23/98]: 0.4503

Mid-Epoch Loss [24/98]: 0.4477

Mid-Epoch Loss [25/98]: 0.4455

Mid-Epoch Loss [26/98]: 0.4429

Mid-Epoch Loss [27/98]: 0.4401

Mid-Epoch Loss [28/98]: 0.4383

Mid-Epoch Loss [29/98]: 0.4359

Mid-Epoch Loss [30/98]: 0.4338

Mid-Epoch Loss [31/98]: 0.4319

Mid-Epoch Loss [32/

  0%|          | 0/11 [00:00<?, ?it/s]

[Epoch 0] Train:  0.3797
Test:  Cos: 0.2006, MSE: 0.3512


  0%|          | 0/98 [00:00<?, ?it/s]

Mid-Epoch Loss [1/98]: 0.3538

Mid-Epoch Loss [2/98]: 0.3486

Mid-Epoch Loss [3/98]: 0.35

Mid-Epoch Loss [4/98]: 0.351

Mid-Epoch Loss [5/98]: 0.3511

Mid-Epoch Loss [6/98]: 0.3497

Mid-Epoch Loss [7/98]: 0.3504

Mid-Epoch Loss [8/98]: 0.348

Mid-Epoch Loss [9/98]: 0.3482

Mid-Epoch Loss [10/98]: 0.3481

Mid-Epoch Loss [11/98]: 0.3483

Mid-Epoch Loss [12/98]: 0.3478

Mid-Epoch Loss [13/98]: 0.3489

Mid-Epoch Loss [14/98]: 0.3499

Mid-Epoch Loss [15/98]: 0.3486

Mid-Epoch Loss [16/98]: 0.3487

Mid-Epoch Loss [17/98]: 0.3483

Mid-Epoch Loss [18/98]: 0.3492

Mid-Epoch Loss [19/98]: 0.3497

Mid-Epoch Loss [20/98]: 0.3502

Mid-Epoch Loss [21/98]: 0.3498

Mid-Epoch Loss [22/98]: 0.3493

Mid-Epoch Loss [23/98]: 0.3498

Mid-Epoch Loss [24/98]: 0.3501

Mid-Epoch Loss [25/98]: 0.3501

Mid-Epoch Loss [26/98]: 0.3501

Mid-Epoch Loss [27/98]: 0.3506

Mid-Epoch Loss [28/98]: 0.3512

Mid-Epoch Loss [29/98]: 0.3512

Mid-Epoch Loss [30/98]: 0.3513

Mid-Epoch Loss [31/98]: 0.3517

Mid-Epoch Loss [32/98

KeyboardInterrupt: 

In [ ]:
import librosa
import IPython
import numpy as np
import torch
import os

from datasets import tqdm
from training.inference import load_and_parse_audio

def test(model):
    path = "E:\\SongsDataset\\songs\\"
    all_folders = os.listdir(path)

    with torch.no_grad():
        for each_song in tqdm(all_folders[100:110]):
            song_path = os.path.join(path, each_song)
            chunks = load_and_parse_audio(song_path, convert=True, chunk_size=1024).to("cuda")
            permuted_chunks = torch.stack([c for c in chunks])

            mean = permuted_chunks.mean(dim=[1, 2], keepdim=True)
            std = permuted_chunks.std(dim=[1, 2], keepdim=True)

            permuted_chunks = (permuted_chunks - mean) / (std + 1e-6)

            reconstructed, latent = model(permuted_chunks)

            input_tensor = np.concatenate(permuted_chunks.cpu().detach().numpy(), axis=1)
            reconstructed = np.concatenate(reconstructed.cpu().detach().numpy(), axis=1)

            input_tensor = input_tensor[:, :512]
            reconstructed = reconstructed[:, :512]

            graph(input_tensor, reconstructed)

            S_recon = librosa.feature.inverse.mel_to_stft(reconstructed)
            Y_recon = librosa.griffinlim(S_recon)

            S_orig = librosa.feature.inverse.mel_to_stft(input_tensor)
            Y_orig = librosa.griffinlim(S_orig)

            IPython.display.display(IPython.display.Audio(Y_orig, rate=44100))
            IPython.display.display(IPython.display.Audio(Y_recon, rate=44100))

In [ ]:
test(model)